# Предподготовка данных тренировочного датасета

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score
import joblib

In [2]:
RANDOM_STATE = 123

In [3]:
np.random.seed(RANDOM_STATE)

In [4]:
df = pd.read_csv('train_dataset_train.csv', low_memory=False)

In [5]:
df.head()

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629,Участок,7503.0,N,0.0,Y,N,87.0,...,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976,ММПО,7503.0,N,0.0,N,N,107.0,...,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962,Цех,7503.0,N,0.0,Y,N,50.0,...,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964,Цех,7503.0,N,0.0,Y,N,416.0,...,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819,Участок,7503.0,N,0.0,Y,N,795.0,...,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   id                             int64  
 1   oper_type + oper_attr          object 
 2   index_oper                     object 
 3   type                           object 
 4   priority                       float64
 5   is_privatecategory             object 
 6   class                          float64
 7   is_in_yandex                   object 
 8   is_return                      object 
 9   weight                         float64
 10  mailtype                       float64
 11  mailctg                        float64
 12  mailrank                       float64
 13  directctg                      float64
 14  transport_pay                  float64
 15  postmark                       float64
 16  name_mfi                       object 
 17  weight_mfi                     float64
 18  pr

Есть строковые признаки. Некоторые переведём в категориальный тип с помощью метода get_dummies. Некоторые удалим.

In [7]:
df.is_privatecategory.value_counts()

N    5994524
Y       2806
0       2670
Name: is_privatecategory, dtype: int64

In [8]:
# пропорции Y и N 
5994524 / (5994524 + 2806), 2806 / (5994524 + 2806)

(0.9995321251290158, 0.0004678748709842547)

в столбце 'is_privatecategory' вместо 0 ставим N и Y с вероятностями (0.999532; 0.000468) 

In [9]:
a = ['Y', 'N']
df['is_privatecategory'].where(df['is_privatecategory'] != '0', other=np.random.choice(a, 6000000, p=[0.000468, 0.999532]),  inplace=True)

In [10]:
df.is_privatecategory.value_counts()

N    5997194
Y       2806
Name: is_privatecategory, dtype: int64

In [11]:
df[['is_privatecategory_0', 'is_privatecategory_1']]  = pd.get_dummies(df['is_privatecategory'])
df.drop('is_privatecategory', axis=1, inplace=True)

In [12]:
df[['mailtype', 'weight']] = df[['mailtype', 'weight']].astype(int)
df[['priority', 'weight']] = df[['priority', 'weight']].astype(int)

In [13]:
df.is_in_yandex.value_counts()

N    3585956
Y    2411374
0       2670
Name: is_in_yandex, dtype: int64

In [14]:
# пропорции Y и N 
3585956 / (3585956+2411374), 2411374 / (3585956+2411374) #  высчитываем пропорции N и Y

(0.597925410140846, 0.402074589859154)

в столбце 'is_in_yandex' вместо 0 ставим Y и N с вероятностями (0.5979254; 0.4020746)

In [15]:
a = ['Y', 'N']
df['is_in_yandex'].where(df['is_in_yandex'] != '0', other=np.random.choice(a, 6000000, p=[0.4020746, 0.5979254]), inplace=True)

In [16]:
df.is_in_yandex.value_counts()

N    3587539
Y    2412461
Name: is_in_yandex, dtype: int64

In [17]:
df[['is_in_yandex_n', 'is_in_yandex_y']] = pd.get_dummies(df.is_in_yandex)
df.drop('is_in_yandex', axis=1, inplace=True)

In [18]:
df.directctg.value_counts()

2.0    5930484
1.0      69361
0.0        155
Name: directctg, dtype: int64

In [19]:
df[['directctg_0', 'directctg_1', 'directctg_2']]  = pd.get_dummies(df.directctg)
df.drop('directctg', axis=1, inplace=True)

In [20]:
df.name_mfi.nunique()

212936

In [21]:
df.index_oper.nunique()

22159

Удаляем столбцы 'name_mfi', 'index_oper'

In [22]:
df.drop('name_mfi', axis=1, inplace=True)
df.drop('index_oper', axis=1, inplace=True)

In [23]:
df.type.value_counts()

Цех               2093668
ММПО              1614974
Участок            926631
ГОПС               463750
ТИ                 305490
П                  245814
МСЦ                223241
СОПС                72161
СЦ                  12469
ОП                   9527
МСО                  7465
ОПП                  7190
АОПП                 6682
МРП                  3850
0                    3444
АО                   1304
ПЕРЕДВИЖНОЕ ОС       1006
ППС                   682
УМСЦ                  569
УКД                    83
Name: type, dtype: int64

In [24]:
df[['type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6', 'type_7', 'type_8', 'type_9', 'type_10', 'type_11', 'type_12', 'type_13', 'type_14', 'type_15', 'type_16', 'type_17', 'type_18', 'type_19', 'type_20']] = pd.get_dummies(df.type)

In [25]:
df.drop('type', axis=1, inplace=True)

In [26]:
df.is_return.value_counts()

N    5993017
Y       6983
Name: is_return, dtype: int64

In [27]:
df[['is_return_0', 'is_return_1']]  = pd.get_dummies(df.is_return)
df.drop('is_return', axis=1, inplace=True)

In [28]:
df['class'].value_counts()

0.0    5214845
3.0     371753
2.0     150393
4.0     131468
1.0     114117
5.0      17424
Name: class, dtype: int64

In [29]:
df[['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5']]  = pd.get_dummies(df['class'])
df.drop('class', axis=1, inplace=True)

In [30]:
df[['pr_7503', 'pr_7504', 'pr_7506', 'pr_0']]  = pd.get_dummies(df['priority'])
df.drop('priority', axis=1, inplace=True)

In [31]:
df['oper_type + oper_attr'].nunique()

109

In [32]:
df.drop('oper_type + oper_attr', axis=1, inplace=True)

In [33]:
df.columns

Index(['id', 'weight', 'mailtype', 'mailctg', 'mailrank', 'transport_pay',
       'postmark', 'weight_mfi', 'price_mfi', 'dist_qty_oper_login_1',
       'total_qty_oper_login_1', 'total_qty_oper_login_0',
       'total_qty_over_index_and_type', 'total_qty_over_index',
       'is_wrong_sndr_name', 'is_wrong_rcpn_name', 'is_wrong_phone_number',
       'is_wrong_address', 'label', 'is_privatecategory_0',
       'is_privatecategory_1', 'is_in_yandex_n', 'is_in_yandex_y',
       'directctg_0', 'directctg_1', 'directctg_2', 'type_1', 'type_2',
       'type_3', 'type_4', 'type_5', 'type_6', 'type_7', 'type_8', 'type_9',
       'type_10', 'type_11', 'type_12', 'type_13', 'type_14', 'type_15',
       'type_16', 'type_17', 'type_18', 'type_19', 'type_20', 'is_return_0',
       'is_return_1', 'class_0', 'class_1', 'class_2', 'class_3', 'class_4',
       'class_5', 'pr_7503', 'pr_7504', 'pr_7506', 'pr_0'],
      dtype='object')

In [34]:
l_columns = list(df.columns)

Столбцов много, поэтому оставим столбцы имеющие хоть какую то корреляцию с целевой переменной

In [35]:
column_corr = []
for feat in l_columns:
     if abs(df['label'].corr(df[feat])) > 0.05:
            print(df['label'].corr(df[feat]), feat)
            column_corr.append(feat)

-0.15113194531159935 mailctg
-0.05501635187474265 transport_pay
0.133341682032662 dist_qty_oper_login_1
0.05587039013139361 total_qty_oper_login_1
0.09973841165138884 total_qty_oper_login_0
0.09522937429476985 total_qty_over_index_and_type
0.06706274887218881 total_qty_over_index
0.13493986781593045 is_wrong_phone_number
0.9999999999999998 label
0.13369071721406078 type_4
0.14300243667788803 type_5
0.06502012755783296 type_14
-0.07068342263999323 type_19
-0.12117097177689684 type_20
-0.10637601880629699 class_0
0.0976843032327197 class_3
0.06673082235787955 class_4
0.07225561108246846 pr_7504
-0.09339252517130853 pr_0


In [36]:
column_corr

['mailctg',
 'transport_pay',
 'dist_qty_oper_login_1',
 'total_qty_oper_login_1',
 'total_qty_oper_login_0',
 'total_qty_over_index_and_type',
 'total_qty_over_index',
 'is_wrong_phone_number',
 'label',
 'type_4',
 'type_5',
 'type_14',
 'type_19',
 'type_20',
 'class_0',
 'class_3',
 'class_4',
 'pr_7504',
 'pr_0']

добавляем 'id'

In [37]:
column_corr = ['id', 'label',
 'mailctg',
 'transport_pay',
 'dist_qty_oper_login_1',
 'total_qty_oper_login_1',
 'total_qty_oper_login_0',
 'total_qty_over_index_and_type',
 'total_qty_over_index',
 'is_wrong_phone_number',
 'type_4',
 'type_5',
 'type_14',
 'type_19',
 'type_20',
 'class_0',
 'class_3',
 'class_4',
 'pr_7504',
 'pr_0']

In [38]:
df = df[column_corr]

## Сохранение предобработанных данных

Сохранение предобработанных данных позволить вернуться к более оптимальному варианту предобработки

In [39]:
# df.to_csv('dataset_train_preprocessing19.csv', sep=',', encoding='utf-8') 

# Построение дерева решений

In [40]:
target = df[['id', 'label']]
feature = df.drop('label', axis=1)

In [93]:
feature.columns

Index(['id', 'mailctg', 'transport_pay', 'dist_qty_oper_login_1',
       'total_qty_oper_login_1', 'total_qty_oper_login_0',
       'total_qty_over_index_and_type', 'total_qty_over_index',
       'is_wrong_phone_number', 'type_4', 'type_5', 'type_14', 'type_19',
       'type_20', 'class_0', 'class_3', 'class_4', 'pr_7504', 'pr_0'],
      dtype='object')

In [41]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.25, random_state=RANDOM_STATE, stratify=target['label'])

In [42]:
target['label'].value_counts()

0    5829020
1     170980
Name: label, dtype: int64

## Определяем веса для атрибута class_weight

In [43]:
1 - target['label'].value_counts() / target.shape[0]

0    0.028497
1    0.971503
Name: label, dtype: float64

В виду того, что компьютерные мощности не позволяли запустить GridSearchCV с широким набором параметров обучение модели проходило пошагово:
- поиск оптимального max_depth
- поиск оптимального min_samples_split
- поиск оптимального min_samples_leaf

In [44]:
best_model = None
best_result = 0
score_TreeClass = []
max_d = 20
for param in range(3,max_d, 3):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=param, class_weight={0: 0.0285, 1: 0.9715})
    model.fit(feature_train.iloc[:, 1:], target_train['label'])
    predictions_test = model.predict(feature_test.iloc[:, 1:])
    print("max_depth =", param, " : ", end='\t')
    result = 0.1 * recall_score(target_test['label'], predictions_test) + 0.9 * roc_auc_score(target_test['label'], predictions_test)
    score_TreeClass.append(result)
    print(result)
    if result > best_result:
        best_model = model # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики на тестовых данных

max_depth = 3  : 	0.7760421641115935
max_depth = 6  : 	0.8680403045230352
max_depth = 9  : 	0.8786141872698187
max_depth = 12  : 	0.8786900236130781
max_depth = 15  : 	0.8778952227996637
max_depth = 18  : 	0.8750673766500916


Лучший промежуточный результат дало решающее дерево с максимальной глубиной 12.

Фискируем данный параметр max_depth=12

Теперь подбираем параметр min_samples_split

In [45]:
best_model = None
best_result = 0
score_TreeClass = []
min_sampl_leaf = [15, 20, 23, 25, 30, 35]
for param in min_sampl_leaf:
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=12, min_samples_split=param, class_weight={0: 0.0285, 1: 0.9715})
    model.fit(feature_train.iloc[:, 1:], target_train['label'])
    predictions_test = model.predict(feature_test.iloc[:, 1:])
    print("sampl_leaf =", param, " : ", end='\t')
    result = 0.1 * recall_score(target_test['label'], predictions_test) + 0.9 * roc_auc_score(target_test['label'], predictions_test)
    score_TreeClass.append(result)
    print(result)
    if result > best_result:
        best_model = model # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики на тестовых данных

sampl_leaf = 15  : 	0.8788853964529206
sampl_leaf = 20  : 	0.8788920874596305
sampl_leaf = 23  : 	0.8788990872661041
sampl_leaf = 25  : 	0.8788963080682287
sampl_leaf = 30  : 	0.8788913672720057
sampl_leaf = 35  : 	0.8788814856795597


Лучший промежуточный результат дало решающее дерево с параметрами: max_depth=12, min_samples_split=23

Фискируем: max_depth=12, min_samples_split=23

Теперь подбираем параметр min_samples_leaf

In [46]:
best_model = None
best_result = 0
score_TreeClass = []
min_sampl_leaf = [10, 15, 20, 25, 30]
for param in min_sampl_leaf:
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=12, min_samples_split=23, min_samples_leaf=param, class_weight={0: 0.0285, 1: 0.9715})
    model.fit(feature_train.iloc[:, 1:], target_train['label'])
    predictions_test = model.predict(feature_test.iloc[:, 1:])
    print("sampl_leaf =", param, " : ", end='\t')
    result = 0.1 * recall_score(target_test['label'], predictions_test) + 0.9 * roc_auc_score(target_test['label'], predictions_test)
    score_TreeClass.append(result)
    print(result)
    if result > best_result:
        best_model = model # сохраняем наилучшую модель
        best_result = result #  сохраняем наилучшее значение метрики на тестовых данных

sampl_leaf = 10  : 	0.8789509687328654
sampl_leaf = 15  : 	0.8789399555649335
sampl_leaf = 20  : 	0.878954573812882
sampl_leaf = 25  : 	0.8789297662081968
sampl_leaf = 30  : 	0.8789178256055


Лучший результат дало решающее дерево с параметрами: max_depth=12, min_samples_split=23, min_samples_leaf=20

### Лучшая модель

In [56]:
best_model # лучшая модель

DecisionTreeClassifier(class_weight={0: 0.0285, 1: 0.9715}, max_depth=12,
                       min_samples_leaf=20, min_samples_split=23,
                       random_state=123)

Сохраняем лучшую модель в файл, для использования предсказания на тестовых данных (при отладке)

In [47]:
# filename = 'result_DTC_d12_20_23_Preproc19.sav'
# joblib.dump(best_model, filename)

Лучшая модель сохранена и может быть использована для сохранения лучшего результата

# Предподготовка данных тестового датасета

In [62]:
df_dataset_test = pd.read_csv('test_dataset_test.csv', low_memory=False) # Чтение тестового датасета

In [63]:
df_dataset_test.head()

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address
0,7815282,8_13,102976,ММПО,7503.0,N,0.0,N,N,85.0,...,489.0,972.0,68766046.0,91123247.0,159889293.0,164927295.0,0,0,0,0
1,8443555,8_2,238753,ГОПС,7503.0,N,4.0,Y,N,21.0,...,186.0,2.0,2895.0,1545.0,4440.0,20623.0,0,1,1,0
2,6352559,1020_-1,618254,ГОПС,7503.0,N,4.0,Y,N,388.0,...,500.0,3.0,2751.0,993.0,3744.0,37817.0,0,0,0,0
3,4921420,1016_-1,102968,ТИ,7503.0,N,0.0,N,N,1203.0,...,0.0,69.0,9379609.0,2267333.0,11646942.0,11759329.0,0,1,0,0
4,1420440,1018_-1,102971,Цех,7506.0,N,0.0,N,N,956.0,...,1267.0,181.0,76138189.0,8747.0,76146936.0,84110459.0,0,1,0,0


In [64]:
df_dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 28 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   id                             int64  
 1   oper_type + oper_attr          object 
 2   index_oper                     object 
 3   type                           object 
 4   priority                       float64
 5   is_privatecategory             object 
 6   class                          float64
 7   is_in_yandex                   object 
 8   is_return                      object 
 9   weight                         float64
 10  mailtype                       float64
 11  mailctg                        float64
 12  mailrank                       float64
 13  directctg                      float64
 14  transport_pay                  float64
 15  postmark                       float64
 16  name_mfi                       object 
 17  weight_mfi                     float64
 18  pr

## Предобработка тестового датасета идет по тому же алгоритму, что и обработка тренировочного датасета

In [65]:
df_dataset_test.is_privatecategory.value_counts()

N    3996480
Y       1820
0       1700
Name: is_privatecategory, dtype: int64

In [66]:
a = ['Y', 'N']
df_dataset_test['is_privatecategory'].where(df_dataset_test['is_privatecategory'] != '0', other=np.random.choice(a, 4000000, p=[0.000455, 0.999545]), inplace=True)

In [67]:
df_dataset_test.is_privatecategory.value_counts()

N    3998180
Y       1820
Name: is_privatecategory, dtype: int64

In [68]:
df_dataset_test[['is_privatecategory_0', 'is_privatecategory_1']]  = pd.get_dummies(df_dataset_test['is_privatecategory'])
df_dataset_test.drop('is_privatecategory', axis=1, inplace=True)

In [69]:
df_dataset_test[['mailtype', 'weight']] = df_dataset_test[['mailtype', 'weight']].astype(int)
df_dataset_test[['priority', 'weight']] = df_dataset_test[['priority', 'weight']].astype(int)

In [70]:
df_dataset_test.is_in_yandex.value_counts()

N    2389898
Y    1608402
0       1700
Name: is_in_yandex, dtype: int64

In [71]:
a = ['Y', 'N']
df_dataset_test['is_in_yandex'].where(df_dataset_test['is_in_yandex'] != '0', other=np.random.choice(a, 4000000, p=[0.40227, 0.59773]), inplace=True)

In [72]:
df_dataset_test.is_in_yandex.value_counts()

N    2390899
Y    1609101
Name: is_in_yandex, dtype: int64

In [73]:
df_dataset_test[['is_in_yandex_n', 'is_in_yandex_y']] = pd.get_dummies(df_dataset_test.is_in_yandex)
df_dataset_test.drop('is_in_yandex', axis=1, inplace=True)

In [74]:
df_dataset_test.directctg.value_counts()

2.0    3953723
1.0      46171
0.0        106
Name: directctg, dtype: int64

In [75]:
df_dataset_test[['directctg_0', 'directctg_1', 'directctg_2']]  = pd.get_dummies(df_dataset_test.directctg)
df_dataset_test.drop('directctg', axis=1, inplace=True)

In [76]:
df_dataset_test.drop('name_mfi', axis=1, inplace=True)
df_dataset_test.drop('index_oper', axis=1, inplace=True)

In [77]:
df_dataset_test.type.value_counts()

Цех               1395154
ММПО              1075996
Участок            619027
ГОПС               308878
ТИ                 203015
П                  164467
МСЦ                149374
СОПС                48117
СЦ                   8310
ОП                   6441
МСО                  5117
АОПП                 4532
ОПП                  4463
МРП                  2520
0                    2249
АО                    807
ПЕРЕДВИЖНОЕ ОС        605
УМСЦ                  433
ППС                   430
УКД                    65
Name: type, dtype: int64

In [78]:
df_dataset_test[['type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6', 'type_7', 'type_8', 'type_9', 'type_10', 'type_11', 'type_12', 'type_13', 'type_14', 'type_15', 'type_16', 'type_17', 'type_18', 'type_19', 'type_20']] = pd.get_dummies(df_dataset_test.type)

In [79]:
df_dataset_test.drop('type', axis=1, inplace=True)

In [80]:
df_dataset_test.is_return.value_counts()

N    3995443
Y       4557
Name: is_return, dtype: int64

In [81]:
df_dataset_test[['is_return_0', 'is_return_1']]  = pd.get_dummies(df_dataset_test.is_return)
df_dataset_test.drop('is_return', axis=1, inplace=True)

In [82]:
df_dataset_test['class'].value_counts()

0.0    3476329
3.0     248562
2.0     100242
4.0      87307
1.0      76204
5.0      11356
Name: class, dtype: int64

In [83]:
df_dataset_test[['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5']]  = pd.get_dummies(df_dataset_test['class'])
df_dataset_test.drop('class', axis=1, inplace=True)

In [84]:
df_dataset_test[['pr_7503', 'pr_7504', 'pr_7506', 'pr_0']]  = pd.get_dummies(df_dataset_test['priority'])
df_dataset_test.drop('priority', axis=1, inplace=True)

In [85]:
df_dataset_test.drop('oper_type + oper_attr', axis=1, inplace=True)

<b> Список столбцов делаем такой же как в обработанном тренировочном датасете (только без столбца 'label')</b>

In [86]:
column_corr

['id',
 'label',
 'mailctg',
 'transport_pay',
 'dist_qty_oper_login_1',
 'total_qty_oper_login_1',
 'total_qty_oper_login_0',
 'total_qty_over_index_and_type',
 'total_qty_over_index',
 'is_wrong_phone_number',
 'type_4',
 'type_5',
 'type_14',
 'type_19',
 'type_20',
 'class_0',
 'class_3',
 'class_4',
 'pr_7504',
 'pr_0']

In [87]:
column_corr_test = ['id',
 'mailctg',
 'transport_pay',
 'dist_qty_oper_login_1',
 'total_qty_oper_login_1',
 'total_qty_oper_login_0',
 'total_qty_over_index_and_type',
 'total_qty_over_index',
 'is_wrong_phone_number',
 'type_4',
 'type_5',
 'type_14',
 'type_19',
 'type_20',
 'class_0',
 'class_3',
 'class_4',
 'pr_7504',
 'pr_0']

In [88]:
df_dataset_test = df_dataset_test[column_corr_test]

In [94]:
df_dataset_test.columns

Index(['id', 'mailctg', 'transport_pay', 'dist_qty_oper_login_1',
       'total_qty_oper_login_1', 'total_qty_oper_login_0',
       'total_qty_over_index_and_type', 'total_qty_over_index',
       'is_wrong_phone_number', 'type_4', 'type_5', 'type_14', 'type_19',
       'type_20', 'class_0', 'class_3', 'class_4', 'pr_7504', 'pr_0'],
      dtype='object')

## Сохранение предобработанных данных¶

Сохранение предобработанных данных позволить вернуться к более оптимальному варианту предобработки

In [2]:
# df.to_csv('dataset_test_preprocessing19.csv', sep=',', encoding='utf-8')

# Предсказание по обученному дереву решений¶

In [3]:
# df = pd.read_csv('dataset_test_preprocessing19.csv', low_memory=False, index_col=0)
# чтение обработанного датасета предыдущего лучшего варианта. Если потребуется

Создаем датасет для отправки решения.

In [90]:
df_test = df_dataset_test[['id']]

In [91]:
df_test.head()

,id
0,7815282
1,8443555
2,6352559
3,4921420
4,1420440


Загружаем обученную модель (при работе с нескольмими файлами)

In [4]:
# filename = 'result_DTC_d12_20_23_Preproc19.sav'

# best_model = joblib.load(filename)

### Предсказание

дописываем столбец с предсказаниями

In [95]:
df_test['label'] = best_model.predict(df_dataset_test.iloc[:, 1:])

C:\Users\igor\AppData\Local\Temp/ipykernel_9612/2296575081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['label'] = best_model.predict(df_dataset_test.iloc[:, 1:])


In [96]:
df_test

,id,label
0,7815282,1
1,8443555,1
2,6352559,1
3,4921420,0
4,1420440,0
...,...,...
3999995,9030800,1
3999996,6275048,1
3999997,4701757,0
3999998,611153,0


### Создаем файл для отправки

In [97]:
df_test.to_csv("pochta_predict_19112022.csv", index=False, line_terminator='\n')

Результат проверки данного файла системой: Score = 0.853381